Original implementation of Contrastive-sc method
(https://github.com/ciortanmadalina/contrastive-sc)

In [1]:
import sys
sys.path.append("..")
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn import metrics

import torch
import torch.nn as nn
import copy
from tqdm.notebook import tqdm
import models
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import st_loss

import h5py
import scipy as sp
import scanpy.api as sc
from collections import Counter
import random
import utils
import loop
import pickle

import train
import os
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [2]:
path = "../"
files = glob2.glob(f'{path}real_data/*.h5')
files = [f[len(f"'{path}real_data"):-3] for f in files]
files = ['worm_neuron_cell']

In [3]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")

In [4]:
df = pd.DataFrame()
lr =0.4
layers = [200, 40, 60]
dropout = 0.9
temperature = 0.07
for dataset in files:

    print(f">>>>> Data {dataset}")
    print("SCZI ", sczi[sczi["dataset"] == dataset]["ARI"].mean())
    for nb_genes in [ 500, 300, 1000, 1500, 2000, 3000, 5000,10000]:

        data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
        X = np.array(data_mat['X'])
        Y = np.array(data_mat['Y'])
        cluster_number = np.unique(Y).shape[0]

        X = train.preprocess(X, nb_genes=nb_genes)

        for run in range(10):
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
            dresults = train.run(X,
                                 cluster_number,
                                 dataset,
                                 Y=Y,
                                 nb_epochs=30,
                                 lr=lr,
                                 temperature=temperature,
                                 dropout=dropout,
                                 evaluate=False,
                                 n_ensemble=1,
                                 layers=layers,
                                 save_to=f"{path}output/real_data/{run}/",
                                 save_pred = False)
            dresults["nb_genes"] = nb_genes
            dresults["dropout"] = dropout
            dresults["layers"] = str(layers)
            dresults["run"] = run
            print(f".", end = "")
            print(f"# {nb_genes},", 
                  dresults.get('COMBINED_kmeans_ari', ""),
                  dresults.get('COMBINED_leiden_ari', ""), dresults.get('kmeans_ari_0',""),
                  dresults.get('leiden_ari_0', ""))
            df = df.append(dresults, ignore_index=True)

            df.to_pickle(f"{path}output/pickle_results/real_data/real_data_worm.pkl")

>>>>> Data worm_neuron_cell
SCZI  0.05077670288621814
(4186, 13488) (4186, 13488) keeping 500 genes
.# 500,   0.11097338059816517 0.08935411871918521
.# 500,   0.15056913992510973 0.13217762685619394
.# 500,   0.15568991402297688 0.1341618541737891
.# 500,   0.13644039397036983 0.11514528097696979
.# 500,   0.14735233247145194 0.13760057109785204
.# 500,   0.1343003318498829 0.11777741800561767
.# 500,   0.16354363301399463 0.13426316186693046
.# 500,   0.14199715566925944 0.10987081861121506
.# 500,   0.1362178490145617 0.11301177344250386
.# 500,   0.1977090292089673 0.1602338337363719
(4186, 13488) (4186, 13488) keeping 300 genes
.# 300,   0.09435868820390092 0.10071804224115775
.# 300,   0.09928876839010391 0.08428479993280678
.# 300,   0.12473251784807703 0.10116754550971337
.# 300,   0.11807973725643857 0.1214856689701669
.# 300,   0.10261126196982398 0.11781917025508135
.# 300,   0.09999911666491491 0.08158937055429835
.# 300,   0.09893553096026957 0.08732497322536599
.# 300,   

In [ ]:
df.groupby("nb_genes").mean()